<a href="https://colab.research.google.com/github/cewgs/Unsupervised-Learning-Project/blob/main/unsupervised_learning_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Run the following cell to clone the project repository and install required dependencies.
After running this cell, all data files and notebooks will be available in the Colab environment

In [4]:
# clone repo if not already cloned and install dependency
import os

if not os.path.exists("Unsupervised-Learning-Project"):
    !git clone https://github.com/cewgs/Unsupervised-Learning-Project.git

%cd Unsupervised-Learning-Project

# Install required external library
!pip install -q gower


In [5]:
# Imports
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score
from sklearn.decomposition import PCA
from sklearn.mixture import GaussianMixture
from sklearn.decomposition import PCA
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.manifold import TSNE
import gower
from scipy.stats import chi2_contingency
import random


#### Survey Data from 2017-2021

In [6]:
data_frames = []

data_paths = [

   'Survey Data/2017.csv',
   'Survey Data/2018.csv',
   'Survey Data/2019.csv',
   'Survey Data/2020.csv',
   'Survey Data/2021.csv',
]

col_renames = {
    '*Are you self-employed?*': 'self_employed',
    '<strong>Are you self-employed?</strong>': 'self_employed',
    'Is your employer primarily a tech company/organization?': 'tech_company',
    'Is your primary role within your company related to tech/IT?': 'tech_related_role',
    'Does your employer provide mental health benefits as part of healthcare coverage?': 'benefits',
    'Does your employer provide mental health benefits as part of healthcare coverage?': 'benefits',
    'Does your employer offer resources to learn more about mental health disorders and options for seeking help?': 'workplace_resources',
    'Have you ever discussed your mental health with your employer?': 'mh_employer_discussion',
    'Have you ever discussed your mental health with coworkers?': 'mh_coworker_discussion',
    'Do you have medical coverage (private insurance or state-provided) that includes treatment of mental health disorders?': 'medical_coverage',
    'Do you currently have a mental health disorder?': 'mental_health',
    'Do you *currently* have a mental health disorder?': 'mental_health',
    'Do you *currently* have a mental health disorder?': 'mental_health',
    'How willing would you be to share with friends and family that you have a mental illness?': 'mh_share',
    'What is your age?': 'age',
    'What is your gender?': 'gender',
    'What country do you *live* in?': 'country',
    'What country do you <strong>live</strong> in?': 'country',
    'Does your employer provide mental health benefits as part of healthcare coverage?	': 'benefits',
}

col_to_keep = col_renames.keys()

In [7]:
# Importing the files
for path in data_paths:
  print("\nReading file: ", path)
  df = pd.read_csv(path)
  print('Shape - default: ', df.shape)

  # Dropping columns > 90% missing values
  max_na_filter = (0.9 * len(df))
  df = df.loc[:,(df.isnull().sum(axis = 0) <= max_na_filter)]
  print('Shape filtered: ', df.shape)

  col_to_drop = [item for item in df.columns if item not in col_to_keep]
  df.drop(columns = col_to_drop, inplace = True, errors = 'ignore')
  print('Shape of column filtered: ', df.shape)

  # Renaming columns across the datasets
  df.drop(columns = col_to_drop, inplace = True, errors = 'ignore')
  df.rename(columns = col_renames, inplace = True, errors = 'ignore')

  data_frames.append(df)


Reading file:  Survey Data/2017.csv


FileNotFoundError: [Errno 2] No such file or directory: 'Survey Data/2017.csv'

In [ ]:
df = pd.concat(data_frames, ignore_index = True)
print('combined data shape: ', df.shape)

In [ ]:
# Overview
df.head()

In [ ]:
# dtypes
df.dtypes

### Data Cleaning

In [ ]:
# number of missing values
display(df.isna().sum().sort_values())
display(print("df shape:", df.shape))

##### Tech related roles
reduce the dataset to tech related roles
In this merged dataset we dont have the exact roles for each employee. So we drop rows with missing values. Some other surveys did include the exact roles which would allow to impute the column "tech_related_role". Further, this dataset only contains employed, not self-employed.

In [ ]:
# drop missing values in tech_related_role
df = df[df['tech_related_role'].notna()]

In [ ]:
df['tech_related_role'].value_counts()

In [ ]:
#remove non tech related roles
df.drop(df[df['tech_related_role'] == 0.0].index, inplace = True) # only tech roles remain

# drop the column 'tech_related_roles'
df.drop(columns = 'tech_related_role', inplace = True)


#### Mental Health

In [ ]:
display(df.isna().sum().sort_values())

In [ ]:
df["mental_health"].value_counts()

#### Gender

In [ ]:
df['gender'].value_counts()

In [ ]:
# marking 25 missing values as "others"
df['gender'] = df['gender'].fillna('Other')

# format the records e.g. male/ Male
df['gender'] = df['gender'].str.lower().str.strip()

In [ ]:
# group different genders into male, female, other

gender_male = ['male', 'm', 'man', 'male/he/him', 'let\'s keep it simple and say \"male\"', 'mostly male', 'masculine', 'identify as male', 'masculino', 'cishet male', 'cis male', 'mail', 'male-ish', 'cis-male', 'male (cis)', 'cis hetero male', 'dude', 'cisgender male', 'male, born with xy chromosoms', 'swm', 'ostensibly male']
gender_female = ['female', 'f', 'woman', 'female, she/her', 'femile', 'female (cis)', 'f, cisgender', 'cisgendered woman', 'femmina', 'cis female', 'cis woman', 'cis-female', 'genderqueer demigirl', 'female (cisgender)', 'my sex is female.', 'femail', 'femalw', 'nonbinary/femme', 'cisgender female', 'she/her/they/them', '*shrug emoji* (f)',  'female/gender non-binary.', 'i identify as female']
gender_other = ['agender', 'nonbinary', 'nb', 'b', '43','non-binary and gender fluid', 'gender non-conforming woman','cis-het male', 'demiguy', 'trans non-binary/genderfluid', 'other', 'afab non-binary', 'sometimes', 'questioning', 'none', 'trans man', 'trans woman', 'trans female', 'non-binary/agender', 'make', 'agender trans woman', 'transfeminine', '\-', 'genderqueer/non-binary', 'non binary', 'contextual', 'agender/genderfluid', 'non-binary', 'genderfluid', 'god king of the valajar', 'uhhhhhhhhh fem genderqueer?', 'transgender', 'genderqueer', 'homem cis']

# transform gender in simpler form
df['gender'] = df['gender'].replace(gender_male, 'Male')
df['gender'] = df['gender'].replace(gender_female, 'Female')
df['gender'] = df['gender'].replace(gender_other, 'Other')

# lets check records now
df['gender'].value_counts()



#### benefits

In [ ]:
# check missing values
df.isna().sum().sort_values()

In [ ]:
df['benefits'].value_counts()

In [ ]:
# 'Not eligible for coverage / NA', equal to'No'

df.loc[df['benefits'] == 'Not eligible for coverage / NA' , 'benefits'] = 'No'

df['benefits'].value_counts()

#### workplace_resources

In [ ]:
df['workplace_resources'].value_counts()

#### mh_employer_discussion

In [ ]:
df['mh_employer_discussion'].value_counts()

#### mh_coworker_discussion

In [ ]:
df['mh_coworker_discussion'].value_counts()

#### medical_coverage

In [ ]:
# check missing values
df.isna().sum().sort_values()

If the the employee has health benifits he should have medical coverage. Further we will use OECD (OECD-ilibrary.org) data to interpolate the coverage for certain countries.

In [ ]:

# medical coverage if the company is providing health benefits
df.loc[df['benefits'] == 'Yes', 'medical_coverage'] = 'Yes'

# Employees are covered (Uk, Germany, Canada, France, Spain, Netherlands)
countries = ['UK', 'Germany', 'Canada', 'France', 'Spain', 'Netherlands']
df.loc[(df['country'].isin(countries)), 'medical_coverage'] = 'Yes'

# Lets check how many null values we have now
df['medical_coverage'].isna().sum()


In [ ]:
# According to OECD, USA has 90% medical coverage
total_us = df.loc[df['country'] == 'United States of America']
no_coverage_us = df.loc[(df['medical_coverage'].isna()) & (df['country'] == 'United States of America')]

print('US employed :{}'.format(len(total_us)))
print('USA residents without medical coverage in df :{}'.format(len(no_coverage_us)))
print('not insured: ' + str(round(100*((len(no_coverage_us)) / (len(total_us))), 2)) + '%')


In [ ]:
# According to the OECD data 90 % in the US should be insured.
# we assign 95 more with insurance.

no_coverage_us_list = list(no_coverage_us.index)

sample = random.sample(no_coverage_us_list, 95) # assign randomly
(sample.sort())

df.loc[sample , 'medical_coverage'] = 'No'
df.loc[(df['country'] == 'United States of America') & (df['medical_coverage'].isna()) , 'medical_coverage'] = 'Yes'

df.isna().sum() / len(df)

In [ ]:
# The remaining 14% will be dropped
df.dropna(inplace = True)

display(df.isna().sum())
display(print("df shape:", df.shape))


#### age

In [ ]:
# We can see outliers as min age is 0.
df['age'].describe()

In [ ]:
# number of unique values per unqiue
df['age'].value_counts()

In [ ]:
# counts with age <= 18
df[(df["age"] <= 18)]

In [ ]:
# Age 0 outlier, drop  outside 21-55 to avoid too small clusters (single counts)
# Drop rows where age <= 21 or age > 55
df = df.drop(df[(df["age"] < 20) | (df["age"] > 55)].index)

# Check summary statistics after dropping
df['age'].describe()


#### mh_share

In [ ]:
# ranges from 0 - 10, ordinal
df['mh_share']

#### formatting

In [ ]:
df['self_employed'] = df['self_employed'].replace({1 : 'Yes' , 0 : 'No'})
df['tech_company'] = df['tech_company'].replace({1.0 : 'Yes' , 0.0 : 'No'})
df['mh_employer_discussion'] = df['mh_employer_discussion'].replace({1.0 : 'Yes' , 0.0 : 'No'})
df['mh_coworker_discussion'] = df['mh_coworker_discussion'].replace({1.0 : 'Yes' , 0.0 : 'No'})

#### self_employed

In [ ]:
#  no self-employed
df.drop(columns = 'self_employed', inplace = True)

#### Region

In [ ]:
# countries
df['country'].value_counts() / len(df)

In [ ]:
# Keep only employers in the USA:
df = df[df['country'] == 'United States of America']
df.shape

### Explorative Data Analysis (USA)





The scope of the explorative analysis are employers working and living in the US.

#### Age Distribution

In [ ]:
# age distrbution
sns.histplot(data=df, x='age', kde=True)
plt.title('Age Distribution')
plt.xlabel('Age')
plt.ylabel('Frequency')

mean_age = df['age'].mean()
median_age = df['age'].median()
std_age = df['age'].std()

plt.text(0.7, 0.9, f'Mean: {mean_age:.2f}', transform=plt.gca().transAxes)
plt.text(0.7, 0.85, f'Median: {median_age:.2f}', transform=plt.gca().transAxes)
plt.text(0.7, 0.8, f'SD: {std_age:.2f}', transform=plt.gca().transAxes)

plt.show()

#### Gender

In [ ]:
colors = sns.color_palette('pastel')[0:5]
df.groupby(['gender']).size().plot(kind = 'pie', autopct = '%1.1f%%',
                                   label = 'Gender', colors = colors)

Most of the samples come from the USA (80%). For explorative analysis we group the countries. The group "others" is a mix of various countries and not meaningful.

In [ ]:
'''
# Group countries into 'Europe', 'United States', and 'Others'
europe_countries = ['United Kingdom', 'Germany', 'Spain', 'France', 'Netherlands', 'Ireland', 'Switzerland', 'Estonia', 'Norway', 'Finland', 'Greece', 'Sweden', 'Poland', 'Portugal', 'Austria']

def categorize_country(country):
    if country == 'United States of America':
        return 'United States'
    elif country in europe_countries:
        return 'Europe'
    else:
        return 'Others'

df['country_group'] = df['country'].apply(categorize_country)

# percentage of individuals with mental health issues within each country group
mental_health_by_country_group = df.groupby('country_group')['mental_health'].value_counts(normalize=True).mul(100).unstack()

display(mental_health_by_country_group)
'''


#### Mental Health Status

In [ ]:
# Mental Health Status overall
df.groupby(['mental_health']).size().plot(kind='pie', autopct='%1.0f%%', label='Mental Health Issues', colors = colors)
plt.show()

#### Mental Health Issues by Gender

In [ ]:
male_with_mental_health = df[(df["mental_health"] == 'Yes') & (df["gender"] == 'Male')]
female_with_mental_health = df[(df["mental_health"] =='Yes') & (df["gender"] == 'Female')]
other_with_mental_health = df[(df["mental_health"] =='Yes') & (df["gender"] == 'Other')]

sizes = [
    male_with_mental_health['gender'].value_counts().get('Male', 0),
    female_with_mental_health['gender'].value_counts().get('Female', 0),
    other_with_mental_health['gender'].value_counts().get('Other', 0),
]
labels = ['Male', 'Female', 'Other']

fig1, ax1 = plt.subplots()
fig1.suptitle('Mental Health Issues by Gender', fontsize=16)
ax1.pie(sizes, labels = labels, autopct = '%1.1f%%', colors = colors)
plt.show()

#### Mental Health by Age and Gender

In [ ]:
# Plot by Age, Gender and Mental Health

g = sns.FacetGrid(df, row = 'gender', col = 'mental_health', height = 4)
g.map(plt.hist, 'age', bins = 10, alpha = 0.6)
g.add_legend()
plt.show()


#### Average/Median Age for Mental Health Status and Gender

In [ ]:
# Mean, median for each gender and their mental health status
average_age_by_gender_and_mh = df.groupby(['gender', 'mental_health'])['age'].agg(['mean', 'median', 'count']).reset_index() # possible features later

# Display the results
display(average_age_by_gender_and_mh)

#### Ressources/Benefits for Mental Health Issues


In [ ]:
# health benefits provided by employer
df.groupby(['benefits']).size().plot(kind = 'pie', autopct = '%1.1f%%', label = 'Health benefits provided ', colors = colors)

In [ ]:
# ressources for mental health provided by employer
df.groupby(['workplace_resources']).size().plot(kind = 'pie', autopct = '%1.1f%%', label = 'Mental Health Resources Provided ', colors = colors)

Following observations can be derived from above.

- More than 60% employee have medical coverage provided from employer, but not the resources to get more information, suggesting that companies do not get active involvement.
- Around 12% employee do not have medical coverage.

#### Mental Health Discussion with Employer


In [ ]:
# Discuss mental health with employer
df.groupby(['mh_employer_discussion']).size().plot(kind = 'pie', autopct = '%1.1f%%', label = 'Discuss Mental Health with Employer ', colors = colors)

In [ ]:
# Employees with mental health issue who discuss it with their employer
male_has_mh_and_not_discussed = df[(df["mental_health"] == 'Yes') & (df["gender"] == 'Male') & (df["mh_employer_discussion"] == 'Yes')]
female_has_mh_and_not_discussed = df[(df["mental_health"] =='Yes') & (df["gender"] == 'Female') & (df["mh_employer_discussion"] == 'Yes')]
other_has_mh_and_not_discussed = df[(df["mental_health"] =='Yes') & (df["gender"] == 'Other') & (df["mh_employer_discussion"] == 'Yes')]

sizes = [
    male_has_mh_and_not_discussed['gender'].value_counts().get('Male', 0),
    female_has_mh_and_not_discussed['gender'].value_counts().get('Female', 0),
    other_has_mh_and_not_discussed['gender'].value_counts().get('Other', 0),
]
labels = ['Male', 'Female', 'Other']

fig1, ax1 = plt.subplots()
fig1.suptitle('Having MH issues and discuss with employer', fontsize=16)
ax1.pie(sizes, labels = labels, autopct = '%1.1f%%', colors = colors)
plt.show()

#### Mental Health Discussion with Coworkers

In [ ]:
df.groupby(['mh_coworker_discussion']).size().plot(kind = 'pie', autopct = '%1.1f%%', label = 'Discuss Mental Health with Co-Workers ', colors = colors)

In [ ]:
male_has_mh_and_not_discussed = df[(df["mental_health"] == 'Yes') & (df["gender"] == 'Male') & (df["mh_coworker_discussion"] == 'Yes')]
female_has_mh_and_not_discussed = df[(df["mental_health"] =='Yes') & (df["gender"] == 'Female') & (df["mh_coworker_discussion"] == 'Yes')]
other_has_mh_and_not_discussed = df[(df["mental_health"] =='Yes') & (df["gender"] == 'Other') & (df["mh_coworker_discussion"] == 'Yes')]

sizes = [
    male_has_mh_and_not_discussed['gender'].value_counts().get('Male', 0),
    female_has_mh_and_not_discussed['gender'].value_counts().get('Female', 0),
    other_has_mh_and_not_discussed['gender'].value_counts().get('Other', 0),
]
labels = ['Male', 'Female', 'Other']

fig1, ax1 = plt.subplots()
fig1.suptitle('Having MH issues and discuss it with coworkers', fontsize=16)
ax1.pie(sizes, labels = labels, autopct = '%1.1f%%', colors = colors)
plt.show()

In [ ]:
# cross-tabulation of mh_employer_discussion and mh_coworker_discussion
discussion_comparison_table = pd.crosstab(df['mh_coworker_discussion'], df['mh_employer_discussion'])

display(discussion_comparison_table)

#### Wllingness Sharing Mental Heatlh with Family/Friends

In [ ]:
# Calculate the median of mh_share
median_mh_share = df['mh_share'].median()
print(f"Median of mh_share: {median_mh_share}")

# Create a histogram of mh_share
plt.hist(df['mh_share'], bins=22, color="skyblue") # ordinal
plt.title('Distribution of Willingness to Share Mental Health with Family/Friends')
plt.xlabel('Willingness to Share (0-10)')
plt.ylabel('Frequency')

# Add a vertical line at the median
plt.axvline(median_mh_share, color='red', linestyle='dashed', linewidth=2, label=f'Median: {median_mh_share:.2f}')
plt.legend()

plt.show()

In [ ]:
# Share Mental Health with Family/Friends
df['mh_share'].value_counts().plot(kind='pie', autopct='%1.1f%%', figsize=(5, 5), colors=sns.color_palette('pastel'))
plt.title('Percentage to Share Mental Health with Family/Friends (0-10)')
plt.ylabel('') # Remove default ylabel
plt.show()

### Feature Engineering



In [ ]:
# dtypes
df.dtypes

#### Binary Encoding

In [ ]:
# store unencoded
df_raw = df.copy()


In [ ]:
df.nunique()

In [ ]:
binary_cols = ['tech_company','mh_employer_discussion','mh_coworker_discussion','medical_coverage']
df[binary_cols] = df[binary_cols].replace({'Yes': 1, 'No': 0})

In [ ]:
df_copy_with_country = df.copy()
df.drop(columns = ['country'], inplace = True)

#### Binning Age

In [ ]:
# Binning into age groups
age_bins = [18, 25, 35, 45, 55]
age_labels = ['18-24','25-34','35-44','45-55']
df['age_groups'] = pd.cut(df['age'], bins=age_bins, labels=age_labels)

#### Binning mh_share

In [ ]:
# binning to low, medium, high
mh_bins = [0, 3, 7, 10]
mh_labels = ['low','medium','high']
df['mh_share_group'] = pd.cut(df['mh_share'], bins=mh_bins, labels=mh_labels)


#### Interaction Feautures

In [ ]:
# Age
df['age_mh_share'] = df['age'] * df['mh_share']
df['age_employer'] = df['age'] * df['mh_employer_discussion']
df['age_coworker'] = df['age'] * df['mh_coworker_discussion']
df['age_tech'] = df['age'] * df['tech_company']

# medical coverage
df['mh_share_employer'] = df['mh_share'] * df['mh_employer_discussion']
df['mh_share_coworker'] = df['mh_share'] * df['mh_coworker_discussion']
df['mh_share_medical'] = df['mh_share'] * df['medical_coverage']
df['mh_share_tech'] = df['mh_share'] * df['tech_company']
df['mh_share_benefits'] = df['mh_share'] * df['benefits']
df['mh_share_resources'] = df['mh_share'] * df['workplace_resources']

# workplace features
df['mh_discussion_both'] = df['mh_employer_discussion'] * df['mh_coworker_discussion']

# Tech company and medical coverage
df['tech_medical'] = df['tech_company'] * df['medical_coverage']

# Tech company and coworker discussion
df['tech_mh_coworker'] = df['tech_company'] * df['mh_coworker_discussion']

# aggregation
# Total discussions
df['mh_discussion_total'] = df['mh_employer_discussion'] + df['mh_coworker_discussion']

# Any discussion flag
df['mh_discussion_any'] = (df['mh_discussion_total'] > 0).astype(int)

# Total resources
df['num_perks'] = df['benefits'] + df['workplace_resources']

# create binary features for specific columns
df['benefits_binary'] = df['benefits'].apply(lambda x: 1 if x == 'Yes' else 'No')
df['workplace_resources_binary'] = df['workplace_resources'].apply(lambda x: 1 if x == 'Yes' else 0)
df['mental_health_binary'] = df['mental_health'].apply(lambda x: 1 if x == 'Yes' else 0)

In [ ]:
df.nunique()

In [ ]:
# Encode 'Yes' and 'No' to 1 and 0 for new features
cols_to_encode = ['benefits_binary','workplace_resources_binary', 'mental_health_binary']
for col in cols_to_encode:
    df[col] = df[col].replace({'Yes': 1, 'No': 0})

df

#### One-hot encoding

In [ ]:
# Get categorical columns
categorical_cols = df.select_dtypes(include='object').columns.tolist()

print("Categorical columns:", categorical_cols)

In [ ]:
df1 = df.copy()

In [ ]:
one_hot_cols = ['benefits','workplace_resources','gender','mental_health','mh_share_group','age_groups','mh_share_benefits',# interaction features
               'mh_share_resources','num_perks'] #,'country_group' without
one_hot_cols2= ['benefits', 'workplace_resources', 'mental_health', 'gender','mh_share_group','age_groups'] # without interaction features

encoded_cols = pd.get_dummies(df1[one_hot_cols], prefix=one_hot_cols, dtype=int)

df2 = pd.concat([df1, encoded_cols], axis=1)
df2.drop(columns=one_hot_cols, inplace=True)
df2.head()

###Feature Selection

In [ ]:
# check unqiue values
# we have some interaction features that will dominate in variance
df.nunique()

#### Chi-Square Test

In [ ]:
# categorical columns
categorical_cols = df.select_dtypes(include='object').columns

print("Chi-Square test:")
print("-" * 50)

# Perform Chi-Square test for each pair of categorical features
for i in range(len(categorical_cols)):
    for j in range(i + 1, len(categorical_cols)):
        col1 = categorical_cols[i]
        col2 = categorical_cols[j]

        # Create a contingency table
        contingency_table = pd.crosstab(df[col1], df[col2])

        # Perform the Chi-Square test
        chi2, p, dof, expected = chi2_contingency(contingency_table)

        print(f"Chi-Square test between '{col1}' and '{col2}':")
        print(f"  Chi2 Statistic: {chi2:.4f}")
        print(f"  P-value: {p:.4f}")
        print(f"  Degrees of Freedom: {dof}")
        print("-" * 30)

#### Scaling

In [ ]:
# Columns to scale with MinMax
cols_to_scale = [
    'mh_share', 'age', 'age_employer', 'age_coworker', 'age_tech',
    'mh_share_employer', 'mh_share_coworker', 'mh_share_medical',
    'age_mh_share', 'mh_share_tech', 'mh_discussion_total'
]

# Binary columns untouched
binary_cols = [col for col in df.columns if df[col].nunique() == 2]

# Columns for MinMax scaling
cols_minmax = [col for col in cols_to_scale if col not in binary_cols]

df_scaled = df2.copy()

# MinMaxScaler
minmax_scaler = MinMaxScaler()
df_scaled[cols_minmax] = minmax_scaler.fit_transform(df_scaled[cols_minmax])

print("Scaled DataFrame:")
display(df_scaled.head())


#### Principal Component Analysis

###### PCA Scaled Data

In [ ]:
# PCA on scaled data
pca = PCA(n_components=2)
X_pca = pca.fit_transform(df_scaled)

# Create a new DataFrame with the PCA components
X_pca_df = pd.DataFrame(data = X_pca, columns = ['pca_1', 'pca_2'])

loadings = pd.DataFrame(pca.components_[0], index=df2.columns, columns=["PC1"])
display(loadings.sort_values("PC1", key=abs, ascending=False))
print("Explained variance ratio:", pca.explained_variance_ratio_)


###### PCA not scaled

In [ ]:
# PCA on not scaled data (df2)
pca = PCA(n_components=2)
X_pca_ns = pca.fit_transform(df2)

# Create a new DataFrame with the PCA components
X_pca_unscaled = pd.DataFrame(data = X_pca, columns = ['pca_1', 'pca_2'])

loadings = pd.DataFrame(pca.components_[0], index=df2.columns, columns=["PC1"])
display(loadings.sort_values("PC1", key=abs, ascending=False))
print("Explained variance ratio:", pca.explained_variance_ratio_)

**If the data is not scaled, the clusters are along age, mh_share (willingness to share with friends). In this case this also includes the interaction features based on these features. The scaled data doesnt provide robust clusters in euclidean space.**

### Unsupervised Model

#### K-Means, Agglomerative Clustering w. PCA, no scaling

In [ ]:
# KMeans
df_ns = X_pca_ns
wcss_kmeans_pca = []
silhouette_scores_kmeans_pca = []
for i in range(1, 11):
    kmeans_pca = KMeans(n_clusters=i, init='k-means++', random_state=42, n_init=10)
    kmeans_pca.fit(df_ns)
    wcss_kmeans_pca.append(kmeans_pca.inertia_)
    if i > 1:
        score = silhouette_score(df_ns, kmeans_pca.labels_)
        silhouette_scores_kmeans_pca.append(score)

# Agglomerative Clustering
silhouette_scores_agg_pca = []
for n_clusters in range(2, 11):
    agg_clustering_pca = AgglomerativeClustering(n_clusters=n_clusters)
    labels_pca = agg_clustering_pca.fit_predict(df_ns)
    score = silhouette_score(df_ns, labels_pca)
    silhouette_scores_agg_pca.append(score)

# Plotting Elbow Method for KMeans
plt.figure(figsize=(10, 5))
plt.plot(range(1, 11), wcss_kmeans_pca, marker='o')
plt.title('Elbow Method')
plt.xlabel('Number of clusters')
plt.ylabel('WCSS')
plt.grid(True)
plt.show()

# Plotting Silhouette Scores for clustering
plt.figure(figsize=(10, 5))
plt.plot(range(2, 11), silhouette_scores_kmeans_pca, marker='o', label='KMeans')
plt.plot(range(2, 11), silhouette_scores_agg_pca, marker='o', label='Agglomerative Clustering')
plt.title('Silhouette Scores')
plt.xlabel('Number of clusters')
plt.ylabel('Silhouette Score')
plt.grid(True)
plt.legend()
plt.show()

#### K-Means & Agglomerative Clustering with PCA/Scaling

In [ ]:
df_pca_reduced = X_pca

# KMeans on PCA reduced data
wcss_kmeans_pca = []
silhouette_scores_kmeans_pca = []
davies_bouldin_scores_kmeans_pca = [] #  Davies-Bouldin
calinski_harabasz_scores_kmeans_pca = [] #  Calinski-Harabasz

K = range(2, 11) # Try a range of cluster numbers

for num_clusters in K:
    kmeans_pca = KMeans(n_clusters=num_clusters, init='k-means++', random_state=42, n_init=10)
    kmeans_pca.fit(df_pca_reduced)
    wcss_kmeans_pca.append(kmeans_pca.inertia_)

    # Calculate evaluation metrics
    labels = kmeans_pca.labels_
    silhouette_scores_kmeans_pca.append(silhouette_score(df_pca_reduced, labels)) # Evaluate on df3
    davies_bouldin_scores_kmeans_pca.append(davies_bouldin_score(df_pca_reduced, labels)) # Added DBI calculation
    calinski_harabasz_scores_kmeans_pca.append(calinski_harabasz_score(df_pca_reduced, labels)) # Added CHI calculation


# Fit KMeans with the chosen number of clusters
# and store the labels in a variable accessible by other cells.
chosen_k = 2 # You can change this based on your preferred number of clusters
final_kmeans_model = KMeans(n_clusters=chosen_k, init='k-means++', random_state=42, n_init=10)
final_kmeans_model.fit(df_pca_reduced)
kmeans_cluster_labels = final_kmeans_model.labels_ # Store labels in a new variable


# Agglomerative Clustering on PCA reduced data
silhouette_scores_agg_pca = []
davies_bouldin_scores_agg_pca = [] # Added Davies-Bouldin scores list
calinski_harabasz_scores_agg_pca = [] # Added Calinski-Harabasz scores list

for n_clusters in range(2, 11):
    # Use 'euclidean' metric for Agglomerative Clustering on PCA reduced data (which is numerical)
    agg_clustering_pca = AgglomerativeClustering(n_clusters=n_clusters, metric='euclidean', linkage='ward') # Using 'ward' linkage as it's common with Euclidean distance
    labels_pca = agg_clustering_pca.fit_predict(df_pca_reduced) # Fit on df3

    # Calculate evaluation metrics
    silhouette_scores_agg_pca.append(silhouette_score(df_pca_reduced, labels_pca)) # Evaluate on df3
    davies_bouldin_scores_agg_pca.append(davies_bouldin_score(df_pca_reduced, labels_pca)) # Added DBI calculation
    calinski_harabasz_scores_agg_pca.append(calinski_harabasz_score(df_pca_reduced, labels_pca)) # Added CHI calculation


# Print the scores for each model and number of clusters
print("KMeans Clustering Scores:") # Updated title
for i, num_clusters in enumerate(K):
    print(f"Number of Clusters (k) = {num_clusters}:")
    print(f"  WCSS: {wcss_kmeans_pca[i]:.2f}")
    if num_clusters > 1:
        print(f"  Silhouette Score: {silhouette_scores_kmeans_pca[i-1]:.4f}") # Adjust index for scores starting from k=2
        print(f"  Davies-Bouldin Index: {davies_bouldin_scores_kmeans_pca[i-1]:.4f}") # Adjust index
        print(f"  Calinski-Harabasz Index: {calinski_harabasz_scores_kmeans_pca[i-1]:.2f}")
    print("-" * 20)

print("\nAgglomerative Hierarchical Clustering Scores:") # Updated title
for i, num_clusters in enumerate(range(2, 11)):
     print(f"Number of Clusters (k) = {num_clusters}:")
     print(f"  Silhouette Score: {silhouette_scores_agg_pca[i]:.4f}")
     print(f"  Davies-Bouldin Index: {davies_bouldin_scores_agg_pca[i]:.4f}")
     print(f"  Calinski-Harabasz Index: {calinski_harabasz_scores_agg_pca[i]:.2f}")
     print("-" * 20)




#### K-Means Clusters (k=4, best model)

In [ ]:
kmeans_model_k4 = KMeans(n_clusters=4,
                         init='k-means++',
                         random_state=42,
                         n_init=10)
kmeans_model_k4.fit(X_pca)


kmeans_labels_k4 = kmeans_model_k4.labels_

print("KMeans model with k=4 fitted successfully on X_pca.")
print("Cluster labels are stored in the 'kmeans_labels_k4' variable.")

# Calculate evaluation metrics for k=4
silhouette_avg = silhouette_score(X_pca, kmeans_labels_k4)
davies_bouldin_avg = davies_bouldin_score(X_pca, kmeans_labels_k4)
calinski_harabasz_avg = calinski_harabasz_score(X_pca, kmeans_labels_k4)

# Print the scores for k=4
print(f"K-Means K = 4")
print(f"  Silhouette Score: {silhouette_avg:.4f}")
print(f"  Davies-Bouldin Index: {davies_bouldin_avg:.4f}")
print(f"  Calinski-Harabasz Index: {calinski_harabasz_avg:.2f}")

###### T-SNE

In [ ]:
# t-SNE for dimensionality reduction
tsne = TSNE(n_components=2,
            perplexity=30,
            learning_rate=200,
            n_iter=1000,
            random_state=42)

tsne_components = tsne.fit_transform(X_pca)

# Create a DataFrame for the t-SNE components and cluster labels
tsne_df = pd.DataFrame(data=tsne_components, columns=['TSNE_1', 'TSNE_2'])
tsne_df['Cluster'] = kmeans_labels_k4

# Visualize the clusters using t-SNE
plt.figure(figsize=(10, 8))
sns.scatterplot(x='TSNE_1', y='TSNE_2', hue='Cluster', data=tsne_df, palette='viridis', legend='full')
plt.title('t-SNE visualization of K-Means (k=4)')
plt.xlabel('TSNE_1')
plt.ylabel('TSNE_2')
plt.show()

###### Cluster Analysis K-Means

In [ ]:
df_with_clusters = df.copy()
df_with_clusters['KMeans Cluster'] = kmeans_labels_k4

# numerical features by cluster
numerical_features_to_analyze = ['age', 'mh_share']
cluster_numerical_means = df_with_clusters.groupby('KMeans Cluster')[numerical_features_to_analyze].mean()

# categorical features by cluster
categorical_features_to_analyze = ['gender', 'mental_health', 'benefits', 'workplace_resources', 'mh_employer_discussion',
                                   'mh_coworker_discussion', 'medical_coverage','age','tech_company','mental_health_binary',
                                   'benefits_binary'] # ,'country_group' exluded

# distribution
cluster_categorical_distribution = {}
for col in categorical_features_to_analyze:
    # frequency and normalize
    distribution = df_with_clusters.groupby('KMeans Cluster')[col].value_counts(normalize=True).mul(100).unstack(fill_value=0)
    cluster_categorical_distribution[col] = distribution

# numerical means
cluster_summary_table = cluster_numerical_means.copy()

# categorical distributions
for col, distribution_df in cluster_categorical_distribution.items():
    # Rename columns
    distribution_df.columns = [f'{col}_{cat}' for cat in distribution_df.columns]

    # summary table
    cluster_summary_table = cluster_summary_table.join(distribution_df)


print("Cluster Summary Table (Mean of Numerical Features and Percentage of Categorical Features):")
display(cluster_summary_table)



for col in categorical_features_to_analyze:
  plt.figure(figsize=(8, 5))
  sns.countplot(data=df_with_clusters, x=col, hue='KMeans Cluster')
  plt.title(f'Distribution of {col} by KMeans Cluster')
  plt.show()

In [ ]:
# Interpretable tables in the cluster table
# binary percentage is 100.0
columns_interpretation = ["age","gender_Male",'gender_Female','gender_Other',"mh_share",'mental_health_Yes',
                          'workplace_resources_Yes','benefits_Yes','tech_company_1', 'mh_coworker_discussion_1',
                          'mh_employer_discussion_1','mental_health_Possibly','mental_health_No'] #,'country_group_United States','country_group_Europe' exluded
cluster_summary_table[columns_interpretation]

In [ ]:
# feature importance
feature_importance1 = cluster_summary_table.max() - cluster_summary_table.min()
feature_importance1.sort_values(ascending=False)

plt.figure(figsize=(40,8))
sns.heatmap(cluster_summary_table, annot=True, cmap='coolwarm')
plt.title("Cluster Centroids Heatmap")
plt.show()



#### Agglomerative with Gower Distance (k=4)

In [ ]:
# Best Model: Agglomerative Clustering with Gower Distance


# Compute Gower distance
gower_dist = gower.gower_matrix(X_pca)

# Agglomerative Clustering with k=4
n_clusters = 4
agg_clustering = AgglomerativeClustering(
    n_clusters=n_clusters,
    metric='precomputed',  #  distance matrix
    linkage='average'
)
labels = agg_clustering.fit_predict(gower_dist)

# evaluation metrics for k=4
silhouette_avg = silhouette_score(gower_dist, labels, metric='precomputed')
davies_bouldin_avg = davies_bouldin_score(X_pca, labels)
calinski_harabasz_avg = calinski_harabasz_score(X_pca, labels)

# scores for k=4
print(f"Agglomerative Hierarchical Clustering Scores (on Gower Distance) for k=4")
print(f"  Silhouette Score: {silhouette_avg:.4f}")
print(f"  Davies-Bouldin Index: {davies_bouldin_avg:.4f}")
print(f"  Calinski-Harabasz Index: {calinski_harabasz_avg:.2f}")

###### Dendogram

In [ ]:
linked = linkage(gower_dist, 'average')

# Plot the dendrogram
plt.figure(figsize=(10, 7))
dendrogram(linked,
            orientation='top',
            distance_sort='descending',
            show_leaf_counts=True)
plt.title('Agglomerative Hierarchical Clustering Dendrogram (Gower Distance)')
plt.xlabel('Sample Index')
plt.ylabel('Distance')
plt.show()

###### T-SNE

In [ ]:
# t-SNE for dimensionality reduction
tsne = TSNE(n_components=2,
            perplexity=30,
            learning_rate=200,
            n_iter=1000,
            random_state=42)

tsne_components = tsne.fit_transform(X_pca)

# DataFrame for the t-SNE components and cluster labels
tsne_df = pd.DataFrame(data=tsne_components, columns=['TSNE_1', 'TSNE_2'])
tsne_df['Cluster'] = labels

# Visualize the clusters
plt.figure(figsize=(10, 8))
sns.scatterplot(x='TSNE_1', y='TSNE_2', hue='Cluster', data=tsne_df, palette='viridis', legend='full')
plt.title('t-SNE visualization of Agglomerative Clustering (k=4)')
plt.xlabel('TSNE_1')
plt.ylabel('TSNE_2')
plt.show()

###### Cluster Analysis 2

In [ ]:
# @title
df_with_clusters2 = df.copy()
df_with_clusters2['Agg_Cluster'] = labels

# numerical features by cluster
numerical_features_to_analyze = ['age', 'mh_share']
cluster_numerical_means = df_with_clusters2.groupby('Agg_Cluster')[numerical_features_to_analyze].mean()

# categorical features by cluster
categorical_features_to_analyze = ['gender', 'mental_health', 'benefits', 'workplace_resources', 'mh_employer_discussion',
                                   'mh_coworker_discussion', 'medical_coverage','age','tech_company','mental_health_binary',
                                   'benefits_binary'] #,'country_group'

# distribution
cluster_categorical_distribution = {}
for col in categorical_features_to_analyze:
    # frequency and normalize
    distribution = df_with_clusters2.groupby('Agg_Cluster')[col].value_counts(normalize=True).mul(100).unstack(fill_value=0)
    cluster_categorical_distribution[col] = distribution

# numerical means
cluster_summary_table2 = cluster_numerical_means.copy()

# categorical distributions to the table
for col, distribution_df in cluster_categorical_distribution.items():
    # Rename columns
    distribution_df.columns = [f'{col}_{cat}' for cat in distribution_df.columns]

    # summary table
    cluster_summary_table2 = cluster_summary_table2.join(distribution_df)


print("Cluster Summary Table 2 (Mean of Numerical Features and Percentage of Categorical Features):")
display(cluster_summary_table2)



for col in categorical_features_to_analyze:
  plt.figure(figsize=(8, 5))
  sns.countplot(data=df_with_clusters2, x=col, hue='Agg_Cluster')
  plt.title(f'Distribution of {col} by  Cluster')
  plt.show()

In [ ]:
# Interpretable tables in the cluster table
# binary percentage is 100.0
columns_interpretation = ["age","gender_Male",'gender_Female','gender_Other',"mh_share",'mental_health_Yes',
                          'workplace_resources_Yes','benefits_Yes','tech_company_1', 'mh_coworker_discussion_1',
                          'mh_employer_discussion_1','mental_health_Possibly'] # ,'country_group_United States','country_group_Europe' exluded
cluster_summary_table2[columns_interpretation]